In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from zipfile import ZipFile
from functools import reduce

import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [ ]:
!pip install -q gdown httpimport
!gdown 'https://github.com/wenjie-hoo/baltic_ml/raw/main/2022.11.07-merged-single-observation.zip'

In [ ]:
zip_file = ZipFile('./2022.11.07-merged-single-observation.zip')
dfs = {text_file.filename: pd.read_csv(zip_file.open(text_file.filename))
       for text_file in zip_file.infolist()
       if text_file.filename.endswith('.csv') and not text_file.filename[37:].startswith('.')}

df_list=[]
for df in dfs.values():
       df['DATE'] = pd.to_datetime(df['DATE'],dayfirst = True)
       df_list.append(df)

df = reduce(lambda left,right: pd.merge(left.drop_duplicates(subset=['DATE','DEPTH']),right.drop_duplicates(subset=['DATE','DEPTH']),on=['DATE','DEPTH'],how='outer'), df_list)
# df.to_csv('xxxxx.csv')
# df.head

In [ ]:
# Fill in missing data
for i in df.columns:
    df[i].fillna(df[i].mean(), inplace=True)

In [ ]:
df.dropna(inplace=True)
fig = plt.figure(figsize=(10,10))
sns.heatmap(df.corr(),vmax=1, annot=True, linewidths=0.5, cbar=False, cmap='YlGnBu',annot_kws={'fontsize':10})
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.title('heatmap',fontsize=10)
plt.show()

In [ ]:
X = df.drop(['DATE','OXY','TP','NO2'],axis=1)
y = df[['OXY','TP','NO2']]
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
x_train['PO4']  = x_train['PO4'] * 0.5
# x_train['TP']  = x_train['TP'] * 0.5
x_train['TN']  = x_train['TN'] * 0.5
x_train['SIO2']  = x_train['SIO2'] * 0.8

In [ ]:
scaler = StandardScaler(with_mean = True, with_std = True)
scaler.fit(x_train)
norm_X = scaler.transform(x_train)
norm_X_test = scaler.transform(x_test)

In [ ]:
# linear regression
lg = LinearRegression()
lg.fit(norm_X, y_train)
y_pred = lg.predict(norm_X_test)
print(mean_squared_error(y_test,y_pred))

In [ ]:
# neural netword
mlp = MLPRegressor(activation='logistic', hidden_layer_sizes=(2,2), solver='sgd', max_iter=3000)
mlp.fit(norm_X,y_train)
y_pred = mlp.predict(norm_X_test)
print(mean_squared_error(y_test, y_pred))

In [ ]:
# random forest 
rfr = RandomForestRegressor(random_state=0)
rfr.fit(norm_X, y_train)
y_pred = rfr.predict(norm_X_test)
print(mean_squared_error(y_test, y_pred))

In [ ]:
feat_lebels = x_train.columns[0:]
importances  = rfr.feature_importances_
indices = np.argsort(importances)[::-1]
index_list = []
value_list = []
for i,j in zip(range(x_train.shape[1]-1),indices):
    index_list.append(feat_lebels[j])
    value_list.append(importances[j])
    print(i+1, feat_lebels[j], importances[j])

plt.figure(figsize=(10,6))
plt.barh(index_list[::-1],value_list[::-1])
plt.yticks(fontsize = 12)
plt.title('feature',fontsize = 14)
plt.show()

In [ ]:
# find best parameters
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators':[150,200,300],
    'min_samples_split':[2,3,5],
    'min_samples_leaf':[1,2,3]
}
grid_seach_rf= GridSearchCV(estimator=RandomForestRegressor(random_state=0),param_grid=param_grid, scoring='neg_mean_squared_error', cv = 5)
grid_seach_rf.fit(norm_X, y_train)
print(grid_seach_rf.best_params_)

In [ ]:
# model
rfr_ = RandomForestRegressor(n_estimators=150, min_samples_leaf=1, min_samples_split=2, random_state=0)
rfr_.fit(norm_X, y_train)
y_pred = rfr_.predict(norm_X_test)
print(mean_squared_error(y_test, y_pred))
y_pred.shape

In [ ]:
# score

from sklearn.metrics import r2_score
regressor = RandomForestRegressor(n_estimators=100, min_samples_leaf=5)
regressor.fit(norm_X, y_train) 
y_pred = regressor.predict(norm_X_test)
print('sklearn score:{}'.format(r2_score(y_test, y_pred)))  

errors = abs(y_pred - y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:\n', np.round(np.mean(errors), 2), 'degrees.')
mape = 100 * (errors / y_test)
accuracy = 100 - np.mean(mape)
print('Accuracy:\n', round(accuracy, 2), '%.')


In [ ]:
col = y_pred.shape[0]
y_pred = pd.DataFrame(y_pred[-col:], index=range(col), columns=['OXY','TP','NO2'])

In [ ]:
plt.figure(figsize=(20, 5))
plt.plot([i for i in range(y_test.shape[0])], y_test['OXY'], color='red', alpha=0.8, label="OXY_test")
plt.plot([i for i in range(y_test.shape[0])], y_pred['OXY'], color='blue', alpha=0.8, label="OXY_pred")
plt.legend(loc="upper right")
plt.title("OXY")
plt.show()

In [ ]:
errors = abs(y_pred - y_test)
print('Mean Absolute Error:\n', np.round(np.mean(errors), 2), 'degrees.')

In [ ]:
x_ = np.arange(col)
plt.plot(x_, df_y_pred['TP'], label = 'tp_pred')
plt.plot(x_,y_test['TP'][-col:], label = 'tp_test')
plt.title('TP')
plt.legend()
plt.show()